In [4]:
import re 
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential

ValueError: Name tf.RaggedTensorSpec has already been registered for class tensorflow.python.ops.ragged.ragged_tensor.RaggedTensorSpec.

In [69]:
def load_data(file_path):
    with open (file_path, "r", encoding="utf-8") as f:
        text = f.read()
    return text

file_path = '/homes/jrgommers/year 3/Deel_B/copilot.txt'
data = load_data(file_path)


In [70]:
def preprocess_text(text):
    text = re.sub(r"[^\w\s]", "", text)  
    text = re.sub(r"\s+", " ", text.strip().lower()) 

    return text

cleaned_data = preprocess_text(data)

unique_chart = sorted(set(cleaned_data))

char_to_index = {char:index for index,char in enumerate(unique_chart)}
index_to_char = {index:char for char,index in char_to_index.items()}

print(index_to_char)

print(f"Raw data: {data[:100]}")  
print(f"Cleaned data: {cleaned_data[:100]}") 
print(f"Unique characters (unique_chart): {unique_chart}")
print(f"Length of unique characters: {len(unique_chart)}")


{0: ' ', 1: '0', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '8', 8: '9', 9: 'a', 10: 'b', 11: 'c', 12: 'd', 13: 'e', 14: 'f', 15: 'g', 16: 'h', 17: 'i', 18: 'j', 19: 'k', 20: 'l', 21: 'm', 22: 'n', 23: 'o', 24: 'p', 25: 'q', 26: 'r', 27: 's', 28: 't', 29: 'u', 30: 'v', 31: 'w', 32: 'x', 33: 'y', 34: 'z', 35: 'â'}
Raw data: Breast Cancer
Breast cancer is one of the most common cancers that affects women and people assigned
Cleaned data: breast cancer breast cancer is one of the most common cancers that affects women and people assigned
Unique characters (unique_chart): [' ', '0', '1', '2', '3', '4', '5', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'â']
Length of unique characters: 36


In [71]:
data_as_int = np.array([char_to_index[char]for char in cleaned_data])

sequence_length = 10
step = 1

sequence = []
targets = []

for i in range (0, len(data_as_int)- sequence_length,step):
    sequence.append(data_as_int[i:i+sequence_length])
    targets.append(data_as_int[i+sequence_length])

x = np.array(sequence)
y = np.array(targets)

# y = to_categorical(y,num_classes = len(unique_chart))

print(x.shape)
print(y.shape)

x_encoded = np.array([to_categorical(n,num_classes=len(unique_chart)) for n in x])
y_encoded = np.array(to_categorical(y,num_classes=len(unique_chart)))

(14433, 10)
(14433,)


In [72]:
from tensorflow.keras import Input
# define model
model = Sequential()
model.add(Input(shape=(sequence_length, len(unique_chart))))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_chart), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='mse')
model.summary()
 

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_18 (LSTM)                  │ (None, 10, 128)        │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 36)             │         4,644 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 220,708 (862.14 KB)

 Trainable params: 220,708 (862.14 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=64,epochs=1)
history = model.fit(x_encoded, y_encoded, batch_size=64,epochs=20)

Epoch 1/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.6696 - loss: 1.1347
Epoch 2/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6959 - loss: 1.0701
Epoch 3/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7156 - loss: 0.9877
Epoch 4/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7361 - loss: 0.9107
Epoch 5/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7570 - loss: 0.8364
Epoch 6/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7793 - loss: 0.7634
Epoch 7/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8039 - loss: 0.6982
Epoch 8/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8283 - loss: 0.6309
Epoch 9/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8359 - loss: 0.5888
Epoch 10/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8523 - loss: 0.5287
Epoch 11/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8711 - loss: 0.4827
Epoch 12/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 

In [76]:
import numpy as np
import random


def generate_sequence(seed_text, model, char_to_index, index_to_char, sequence_length, num_chars_to_generate):

    generated = seed_text
    input_sequence = [char_to_index[char] for char in seed_text]
    print(index_to_char)



    for _ in range(num_chars_to_generate):
        # Zorg ervoor dat de invoer de juiste lengte heeft
        input_sequence_padded = np.array([to_categorical(input_sequence, num_classes=len(unique_chart))])
        
        # Maak een voorspelling
        predicted_probs = model.predict(input_sequence_padded, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        
        # Voeg het voorspelde karakter toe
        predicted_char = index_to_char[predicted_index]
        generated += predicted_char
        input_sequence.append(predicted_index)
    
    return generated



seed_text = "breast cancer"  
num_chars_to_generate = 200  
generated_text = generate_sequence(seed_text, model, char_to_index, index_to_char, sequence_length, num_chars_to_generate)

print("\nGenerated Text:")
print(generated_text)



{0: ' ', 1: '0', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '8', 8: '9', 9: 'a', 10: 'b', 11: 'c', 12: 'd', 13: 'e', 14: 'f', 15: 'g', 16: 'h', 17: 'i', 18: 'j', 19: 'k', 20: 'l', 21: 'm', 22: 'n', 23: 'o', 24: 'p', 25: 'q', 26: 'r', 27: 's', 28: 't', 29: 'u', 30: 'v', 31: 'w', 32: 'x', 33: 'y', 34: 'z', 35: 'â'}

Generated Text:
breast cancer and celone in the the the comeconcer cancer cancer cancer that celliding of your coloncancerally affecticenes whanges whangs of your blood cancerous celliding on there of the that cancer that cancero
